In [3]:
from tqdm import tqdm
import numpy as np
import MDAnalysis as mda

import pandas as pd
from pathlib import Path
import MDAnalysis.transformations as trans
import seaborn as sns
import os
import sys
import matplotlib.pyplot as plt
from dotenv import load_dotenv
load_dotenv()

from scipy.spatial import KDTree
from scipy.spatial import distance

from utils.classes import Peptide

In [9]:
step_size = 500 
membrane_lipids = 512
peptide_name = "WF2_8_1"

peptide = Peptide(
    pep_name=peptide_name,
    xtc_file_path="/Volumes/miru_back/young_concentration_WF2/WF2_8/md_0_1_combined_first500ns_pbc.xtc",
    tpr_file_path="/Volumes/miru_back/young_concentration_WF2/WF2_8/md_0_1.tpr",
    peptide_number=8, amino_acid_count=25, step_size=step_size)

In [10]:
def get_index_shortest_distance(point_p, b):

    point_p = point_p[0]
    distances_array = np.empty(len(b))

    for i in range(len(b)):
        p = b[i]
        distances_array[i] = np.sqrt((point_p[1]-p[1])**2+(point_p[0]-p[0])**2+(point_p[2]-p[2])**2)
    distances_array = distances_array[distances_array!=0]

    return(b[np.argmin(distances_array)], np.min(distances_array))

def get_closest_lipid_z(res_pos, lipids, p_up, p_low):
    all_up = lipids
    closest_lipid_z = None
    kdtree=KDTree(lipids)
    
    for radius in [(float(p_up) - float(p_low))/2, (float(p_up) - float(p_low)), 20, 40, 80, 120]:
        
        x = kdtree.query_ball_point(res_pos, radius, return_sorted=True)
        
        if len(x[0])==0:
            continue
        else:
            closest_lipid_index, distance_closest_lipid = get_index_shortest_distance(res_pos, all_up[x[0]])
            return closest_lipid_index[2]

In [17]:

u = peptide.u
z_pos_list = []
frames, n_frames = peptide.load_traj()
for frame_index, ts in tqdm(enumerate(u.trajectory[frames]), total=n_frames): 
    u.select_atoms(f'resname POPG and name P').positions
    p_memb = u.select_atoms(f'resname POPG and name P').positions
    p_up = np.mean(p_memb[:int(membrane_lipids/2)])
    p_low = np.mean(p_memb[int(membrane_lipids/2):])

    for pep, res_id_range in peptide.pep_dict.items():
        res_count = 1
        for res_id in range(res_id_range[0], res_id_range[1]+1):
            # Calculate the z pos of each residue one at a time
            res_z_pos = np.mean(u.select_atoms(f'name CA and resid {res_id}').positions[:,[2]].astype(float))
            res_pos = u.select_atoms(f'name CA and resid {res_id}').positions


            # Create list that contains which peptide the residue is in e.g. 1 if it's in peptide 1 etc
            p = []

            # Determine the actual insertion of each residue
            pbc_crossed = 1 if res_z_pos > 0 and res_z_pos < u.dimensions[2]/2 else 0 #This assumes that peptides don't insert more than half than the membrane height

            if pbc_crossed == 0:
                lipid_z = get_closest_lipid_z(res_pos, p_memb[:int(membrane_lipids/2)], p_up, p_low)

                if lipid_z:
                    pep_insertion = res_z_pos - lipid_z
                    
                    z_pos_list.append((u.trajectory.time, pep , res_id, res_count,  pep_insertion, p_up,  p_low, lipid_z, res_z_pos,  pbc_crossed))
                    res_count += 1

df = pd.DataFrame(z_pos_list, columns=["Time (ns)", "Peptide_num", "Resid", "Residue", "CA Z position", "P_up", "P low", "lipid_z", "res_z_pos", "Crossed"])
df['Residue']= df['Residue'].astype('str')
df['Time (ns)'] = df['Time (ns)'].astype(float)/1000
df['Time (ns)'] = df['Time (ns)'].astype(int)
df.to_csv(f"zpos_{peptide.pep_name}_insertion_curv_100.csv")

100%|██████████| 51/51 [01:51<00:00,  2.18s/it]
